In [1]:
import clip
import torch
import torch.nn.functional as F
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from types import MethodType
import os
import zipfile
import requests
from io import BytesIO
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


/opt/anaconda3/envs/wb-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def wrap_vit_blocks(model):
    original_blocks = model.visual.transformer.resblocks
    activations = {}

    for i, block in enumerate(original_blocks):
        def make_custom_forward(orig_block, layer_name):
            def custom_forward(self, x):
                out = orig_block(x)
                activations[layer_name] = out.clone()
                return out
            return custom_forward

        block.forward = MethodType(make_custom_forward(block.forward, f"layer_{i}"), block)

    return activations



def logit_lens_analysis(activations, projection_head, ln_post, final_output, text_features, dictionary):
    '''
    Perform logit lens analysis on the activations
    Returns:
        - distances: cosine similarity to final output
        - predictions: (predicted_label, similarity_score)
    '''
    distances = {}
    predictions = {}

    for name, x in activations.items():
        # x shape: (seq_len, batch, dim)
        x = x.permute(1, 0, 2)  # -> (batch, seq_len, dim)
        cls_token = x[:, 0, :]  # take CLS token

        # Apply final layer norm
        cls_token = ln_post(cls_token)

        # Project using CLIP's final projection matrix
        projected = cls_token @ projection_head  # (1, 512)
        projected = F.normalize(projected, dim=-1)

        # Cosine similarity with final output
        similarity = F.cosine_similarity(projected, final_output, dim=-1)
        distances[name] = similarity.item()

        # Cosine similarity with all text features
        text_similarity = F.cosine_similarity(projected, text_features, dim=-1)
        predicted_idx = torch.argmax(text_similarity).item()
        predicted_label = dictionary[predicted_idx]
        predicted_score = text_similarity[predicted_idx].item()

        predictions[name] = (predicted_label, predicted_score)

    return distances, predictions


import os
import csv

def perform_logit_lense_analysis(model, dataset, device):
    idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}
    model.eval()

    # Utwórz folder na wyniki
    os.makedirs("logit_lens_results", exist_ok=True)

    # Rejestracja hooków
    activations = wrap_vit_blocks(model)
    headers = sorted([f"layer_{i}" for i in range(len(model.visual.transformer.resblocks))], key=lambda x: int(x.split('_')[1]))

    # Przygotuj pliki CSV z nagłówkami
    cosine_path = "logit_lens_results/cosine_similarity.csv"
    preds_path = "logit_lens_results/predictions.csv"

    if not os.path.exists(cosine_path):
        with open(cosine_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Image"] + headers)

    if not os.path.exists(preds_path):
        with open(preds_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Image"] + [f"{h}_label" for h in headers] + [f"{h}_prob" for h in headers])

    for image_idx, (image, label) in enumerate(dataset):
        image = image.unsqueeze(0).to(device)
        class_name = idx_to_class[label]
        text = clip.tokenize([f"a photo of a {class_name}"]).to(device)

        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)

        final_output = F.normalize(image_features, dim=-1)

        distances, predictions = logit_lens_analysis(
            activations,
            model.visual.proj,
            model.visual.ln_post,
            final_output,
            text_features,
            idx_to_class  # słownik klas jako dictionary
        )

        # --- Zapis cosine similarity ---
        cosine_row = [f"Image_{image_idx + 1}"] + [distances[layer] for layer in headers]
        with open(cosine_path, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(cosine_row)

        # --- Zapis predykcji i prawdopodobieństw ---
        pred_labels = [predictions[layer][0] for layer in headers]
        pred_probs = [predictions[layer][1] for layer in headers]
        pred_row = [f"Image_{image_idx + 1}"] + pred_labels + pred_probs
        with open(preds_path, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(pred_row)



def plot_results(distances, predictions):
    '''
    Plots the results of the logit lens analysis
    '''
    layer_names = sorted(distances.keys(), key=lambda x: int(x.split('_')[1]))
    similarity_values = [distances[layer] for layer in layer_names]
    predicted_labels = [predictions[layer] for layer in layer_names]

    plt.figure(figsize=(10, 6))
    sns.heatmap(np.array(similarity_values).reshape(1, -1), annot=True, cmap="viridis", xticklabels=layer_names, yticklabels=["Cosine Similarity"], cbar_kws={'label': 'Cosine Similarity'}, cbar=True)

    for i, label in enumerate(predicted_labels):
        plt.text(i + 0.5, 0, label, ha='center', va='center', color='black', fontsize=10, rotation=90)

    plt.title("Cosine Similarity Heatmap for Each Layer with Predictions")
    plt.xlabel("Layer")
    plt.tight_layout()
    plt.show()

In [3]:
# URL i ścieżka tymczasowa
url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
extract_path = "./tiny-imagenet-200"

print("Downloading Tiny ImageNet...")
response = requests.get(url)
with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
    zip_ref.extractall(".")

print("Download and extraction complete.")


train_dir = os.path.join(extract_path, "train")
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                        std=[0.26862954, 0.26130258, 0.27577711]),
])
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)



Download and extraction complete.


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load CLIP ViT model
model, preprocess = clip.load("ViT-B/32", device=device)
model.eval()

In [7]:
perform_logit_lense_analysis(model=model, dataset=train_dataset, device=device)